In [1]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
def getStateAtMoment(base,distance,sinal,inst):
    inst = inst+(distance/sinal)
    if inst >= 1:
        if(inst%round(inst)>0):
            inst=round(inst)+1
        else:
            inst=round(inst)
    else:
        return base,round(inst)
    if inst%2==0:
        return base,inst
    else: 
        base = not base
        return base,inst

In [3]:
def DecomposeRoute(Solution,veiculos,vitimas):
    routes = {x:{} for x in veiculos.index}  
    #Separa as rotas de cada veículo
    for i in Solution:
            aux = i.split("_")[1:]
            routes[int(aux[2])][int(aux[0])]=int(aux[1])
        
    ordered_routes = {x:[] for x in veiculos.index}  
    for k in veiculos.index:
        if len(routes[k]) > 0:

            inicial = veiculos.loc[k]["Inicial"]
            actual_node = routes[k].pop(inicial)

            ordered_routes[k].append(inicial)
            
            if len(routes[k]) == 0:
                ordered_routes[k].append(actual_node)
            else:
                while actual_node not in vitimas["Ponto"].values:
                    
                    ordered_routes[k].append(actual_node)
                    
                    next_node = routes[k].pop(actual_node)

                    actual_node=next_node
                
                ordered_routes[k].append(actual_node)
    return ordered_routes

In [4]:
def makeArcs(ordered_routes,_DistanceMatrix,veiculos,vitimas,atendimento,pontos,estado,sinal):
    arc_time = {}
    arc_states = {}
    arc_moment={}
    arc_priority = {}

    for k in range(len(ordered_routes)):
        soma_tempo=0
        moment = 0
        
        for i in range(len(ordered_routes[k])-1):
            saida = ordered_routes[k][i]
            chegada = ordered_routes[k][i+1]
            
            soma_tempo+=_DistanceMatrix.loc[saida][chegada]
            
            index_position = pontos.loc[saida][2:][pontos.loc[saida][2:] == chegada].index[0]

            state,moment = getStateAtMoment(estado.loc[saida][index_position],
                                 _DistanceMatrix.loc[saida][chegada],
                                 sinal,moment)
            
            if (saida,chegada) not in arc_time.keys():
                arc_time[(saida,chegada)] = soma_tempo
                arc_moment[(saida,chegada)] = moment
                arc_states[(saida,chegada)] = state
            
            elif arc_time[(saida,chegada)]<soma_tempo:
                arc_time[(saida,chegada)] = soma_tempo
                arc_moment[(saida,chegada)] = moment
                arc_states[(saida,chegada)] = state
        
            priority = vitimas[vitimas["Ponto"] == atendimento[k]]["Prioridade"].values[0]
            
            if (saida,chegada) not in arc_priority.keys():
                arc_priority[(saida,chegada)] = priority

            elif arc_priority[(saida,chegada)] < priority:
                arc_priority[(saida,chegada)] = priority

        
    return arc_time,arc_states,arc_moment,arc_priority